In [1]:
from selenium import webdriver 
import pandas as pd 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
driver = webdriver.Chrome('/Applications/chromedriver')
driver.get('https://music.youtube.com/moods_and_genres')
soup = BeautifulSoup(driver.page_source)
button_driver = driver.find_elements_by_xpath('//button[@class="style-scope ytmusic-navigation-button-renderer"]')
button_ls = soup.find_all('button', class_='style-scope ytmusic-navigation-button-renderer')
mood_playlist = {}
for i in range(len(button_driver)):
    page = driver.find_elements_by_xpath('//button[@class="style-scope ytmusic-navigation-button-renderer"]')[i]
    # go to next page and wait for computer to load
    new = page.click()
    time.sleep(3)
    # get the json in second page
    subhtml = driver.page_source
    soup2 = BeautifulSoup(subhtml,'html.parser')
    mood = soup2.find(class_='title style-scope ytmusic-header-renderer').text
    # scrap all playlist link from each category
    playlist_link = ['https://music.youtube.com/' + i['href'] for i in soup2.find_all(class_='yt-simple-endpoint style-scope yt-formatted-string')]
    # use a dict to store all playlists per mood
    mood_playlist[mood] = playlist_link
    driver.back()
    subsubhtml = driver.page_source
driver.close()

In [2]:
mood_playlist

{'Black Lives Matter': ['https://music.youtube.com/playlist?list=RDCLAK5uy_k73Q_g6EpJb3aVOv2aj-Wper78yWGV6xI',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_mase_R11jkzHUenhJxzw4SRTZg4isU1yg',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_nlvLeSaigisz6SiQZBdGa7O7GPTYWBQ8E',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_nzTmjsg0fFvlPwCGkjlM_NNFykrYjAaOA',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_k0JkYgFhsrwXXxJVkUtl5LG3Nv8pFwWhQ',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_nGgzkCTUJrY6A8IWiySTH-NK_7dNOB5Bk',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_miMSxuvE6h6QxKk8pnABxR-CbiRcA5KiI',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_nwcZyv_KJHkCFG-k27Xkw64ZdXMXadhx0',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_lHiUUBX_lSTd22ATDZV77s6HyL3FunfJA',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_ktpdBadVXF5AjZxlIptFEnDtBFUiBKz74',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_ksq1FGO3ohfB2RKcYI5x4IDy4NP7AxDbY']

In [3]:
# Manually filter out some unrelated category
# mood type in mood_ls is believed to be some regular mood available on youtube music > so i just picked these
mood_ls = ['Chill','Energy Boosters','Feel Good','Party','Classical','Dance & Electronic','Decades','HipHop','Jazz','Pop','R&B & Soul','Rock']
final_playlist = {}
for cat, ls in mood_playlist.items():
    if cat in mood_ls:
        final_playlist[cat] = ls
final_playlist        

{'Chill': ['https://music.youtube.com/playlist?list=RDCLAK5uy_nHSqCJjDrW9HBhCNdF6tWPdnOMngOv0wA',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_kluqSMlnog1GDp3fK-kzf787f9DMW8LB0',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_nyKVppE-RpLkeCcwLct4rvN9e8AAsS_qw',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_lBhJGJYtUsZjFHe5US4EXaHcZltRaIzvM',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_njvsGKIUycy_a4h7zTS8upbKhHcMVzHFM',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_kqQ1zj3L4fZK6ye75srlXMLYXdh2jMFPQ',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_nM6gXW5m1DqYCO6eu0P8Di2jEdj_Hm4fQ',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_nPuTx5ypLstZav2z7cQKkPHt6bKOM_MZA',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_kT-sIJz2O-hpkxwjosN2hMt9Y5xevcPYI',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_lNJA7PB9DAQEdtTnfuKaC2XEOAE1OoX50',
  'https://music.youtube.com/playlist?list=RDCLAK5uy_lBcYC0xVqLIxqJ6Ug0Hmj8iJNWvbKcRQA',
  'https://m

In [4]:
df = pd.DataFrame(final_playlist.items(), columns = ['mood','final_playlist'])

In [5]:
ls = df.mood.to_list()

In [6]:
ls

['Chill',
 'Energy Boosters',
 'Feel Good',
 'Party',
 'Classical',
 'Dance & Electronic',
 'Decades',
 'Jazz',
 'Pop',
 'R&B & Soul',
 'Rock']

In [7]:
# categorise mood into positive, negative, neutural
def mood(x):
    if x in ['Chill','Energy Boosters', 'R&B & Soul', 'Rock']:
        mood = 'negative'
    elif x in ['Classical','Decades','Jazz']:
        mood = 'neutral'
    else:
        mood = 'positive'
    return mood

df['mood_assignment'] = df['mood'].apply(mood)
df

,mood,final_playlist,mood_assignment
0,Chill,[https://music.youtube.com/playlist?list=RDCLA...,negative
1,Energy Boosters,[https://music.youtube.com/playlist?list=RDCLA...,negative
2,Feel Good,[https://music.youtube.com/playlist?list=RDCLA...,positive
3,Party,[https://music.youtube.com/playlist?list=RDCLA...,positive
4,Classical,[https://music.youtube.com/playlist?list=RDCLA...,neutral
5,Dance & Electronic,[https://music.youtube.com/playlist?list=RDCLA...,positive
6,Decades,"[https://music.youtube.com/moods_and_genres, h...",neutral
7,Jazz,[https://music.youtube.com/playlist?list=RDCLA...,neutral
8,Pop,[https://music.youtube.com/playlist?list=RDCLA...,positive
9,R&B & Soul,[https://music.youtube.com/playlist?list=RDCLA...,negative


In [8]:
# convert to csv for action.py > need to upload this csv to gcp
df.to_csv('youtube_playlist.csv', index = False)